<a href="https://colab.research.google.com/github/aJimenez19037/GeometricFoundationsProject/blob/main/Copy_of_Safe_ReinforcementLearning_baseline_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Problem Introduction: Safe Reinforcement Learning

<img src="https://raw.githubusercontent.com/eleurent/highway-env/gh-media/docs/media/roundabout-env.gif"></img>


**Optimal Planning with Oracle Model**
When planning with the true model, an optimal policy generally operates as close as possible to the system constraints, resulting in dangerous behaviours.

Example of unsafe behaviour:
<a href="https://imgur.com/o5JvJXi"><img src="https://i.imgur.com/o5JvJXi.png" title="source: imgur.com" /></a>

When turning the green car takes a right but does not stay on the extreme right of the road. It follows the policy to not get hit but this is not safe since other cars may not behave optimally always.

As an effect, even slight model errors can lead to catastrophic failures

<a href="https://imgur.com/sIkfUCC"><img src="https://i.imgur.com/sIkfUCC.png" title="source: imgur.com" /></a>
  
In order to account for model uncertainty, we follow the robust control framework:

"*Maximise the worst-case performance with respect to a set of possible behavioral models*"

**Robust Planning with Discrete Ambiguity**

You can plan by considering every possible direction the traffic participants can take at their next intersection.


**Reducing uncertainty of the next action of other cars**

Let us discuss the reasons behind the uncertainty

1. Driver on the road is not an optimal driver.
2. We do not know the exact location of the driver.

<!-- **Handling Scenario 1:**
We plan by considering every possible direction that traffic participants can take at their next intersection.

$\mu$- mean path of each participant.

<a href="https://imgur.com/hARWjSD"><img src="https://i.imgur.com/hARWjSD.png" title="source: imgur.com" /></a>

Continuous range of trajectories (variance over) path trajectories of each traffic participant based on their driving style.

<a href="https://imgur.com/qd3kyZ0"><img src="https://i.imgur.com/qd3kyZ0.png" title="source: imgur.com" /></a> -->

To summarise, if everyone has the same driving behaviour we can assume drivers to behave as if they are driving on the optimal path. If we also think about each one having a driving behaviour i.e (some can be drunk, talking on phone, angry) then we get varying levels of uncertainty around the optimal path for each vehicle. This is accurately characterized below.

<a href="https://imgur.com/qfDGiTx"><img src="https://i.imgur.com/qfDGiTx.png" title="source: imgur.com" /></a>



# Model-Based Reinforcement Learning

We first demonstrate a Model based approach and its failure cases and then moivate you to explore inverse reinforcement learning approaches. You can choose either of the above approaches.

## Principle for Model based RL
We consider the optimal control problem of an MDP with a **known** reward function $R$ and subject to **unknown deterministic** dynamics $s_{t+1} = f(s_t, a_t)$:

$$\max_{(a_0,a_1,\dotsc)} \sum_{t=0}^\infty \gamma^t R(s_t,a_t)$$

In **model-based reinforcement learning**, this problem is solved in **two steps**:
1. **Model learning**:
We learn a model of the dynamics $f_\theta \simeq f$ through regression on interaction data.
2. **Planning**:
We leverage the dynamics model $f_\theta$ to compute the optimal trajectory $$\max_{(a_0,a_1,\dotsc)} \sum_{t=0}^\infty \gamma^t R(\hat{s}_t,a_t)$$ following the learnt dynamics $\hat{s}_{t+1} = f_\theta(\hat{s}_t, a_t)$.

(We can easily extend to unknown rewards and stochastic dynamics, but we consider the simpler case in this notebook for ease of presentation)


## Motivation

### Sparse rewards
* In model-free reinforcement learning, we only obtain a reinforcement signal when encountering rewards. In environment with **sparse rewards**, the chance of obtaining a reward randomly is **negligible**, which prevents any learning.
* However, even in the **absence of rewards** we still receive a **stream of state transition data**. We can exploit this data to learn about the task at hand.

### Complexity of the policy/value vs dynamics:
Is it easier to decide which action is best, or to predict what is going to happen?
* Some problems can have **complex dynamics** but a **simple optimal policy or value function**. For instance, consider the problem of learning to swim. Predicting the movement requires understanding fluid dynamics and vortices while the optimal policy simply consists in moving the limbs in sync.
* Conversely, other problems can have **simple dynamics** but **complex policies/value functions**. Think of the game of Go, its rules are simplistic (placing a stone merely changes the board state at this location) but the corresponding optimal policy is very complicated.

Intuitively, model-free RL should be applied to the first category of problems and model-based RL to the second category.

### Inductive bias
Oftentimes, real-world problems exhibit a particular **structure**: for instance, any problem involving motion of physical objects will be **continuous**. It can also be **smooth**, **invariant** to translations, etc. This knowledge can then be incorporated in machine learning models to foster efficient learning. In contrast, there can often be **discontinuities** in the policy decisions or value function: e.g. think of a collision vs near-collision state.

###  Sample efficiency
Overall, it is generally recognized that model-based approaches tend to **learn faster** than model-free techniques (see e.g. [[Sutton, 1990]](http://papersdb.cs.ualberta.ca/~papersdb/uploaded_files/paper_p160-sutton.pdf.stjohn)).

### Interpretability
In real-world applications, we may want to know **how a policy will behave before actually executing it**, for instance for **safety-check** purposes. However, model-free reinforcement learning only recommends which action to take at current time without being able to predict its consequences. In order to obtain the trajectory, we have no choice but executing the policy. In stark contrast, model-based methods a more interpretable in the sense that we can probe the policy for its intended (and predicted) trajectory.

## Our challenge: Robust

We consider the **roundabout-v0** task of the [highway-env](https://github.com/eleurent/highway-env) environment. It is a **continuous control** task where an agent **drives a car** by controlling the gaz pedal and steering angle and must **navigate safely** with the appropriate heading.



###  Warming up
We start with a few useful installs and imports:

In [ ]:
# Install environment and visualization dependencies
!pip install highway-env

# Environment
import gymnasium as gym
import highway_env

# Models and computation
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from collections import namedtuple

# Visualization
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm.notebook import trange

# IO
from pathlib import Path

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 23.2 MB/s eta 0:00:00


We also define a simple helper function for visualization of episodes:

In [ ]:
import sys
from tqdm.notebook import trange
!pip install tensorboardx gym pyvirtualdisplay
!apt-get install -y xvfb ffmpeg
!git clone https://github.com/Farama-Foundation/HighwayEnv.git 2> /dev/null

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.1 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common
The following NEW packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common xvfb
0 upgraded, 9 newly installed, 0 to remove and 6 not upgraded.
Need to get 7,814 kB of archives.
After this operation, 11.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfontenc1 amd64 1:1.1.4-1build3 [14.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxfont2 amd64 1:2.0.5-1build1 [94.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxkbfile1 am

In [ ]:
%cd HighwayEnv/scripts

/content/HighwayEnv/scripts


In [ ]:
from utils import record_videos, show_videos

### Let's try it!

Make the environment, and run an episode with random actions:

In [ ]:
env = gym.make("roundabout-v0", render_mode="rgb_array")
env = record_videos(env)
env.reset()
done = False
while not done:
    action = env.action_space.sample()
    obs, reward, done, truncated, info = env.step(action)
env.close()
show_videos()

Moviepy - Building video /content/HighwayEnv/scripts/videos/rl-video-episode-0.mp4.
Moviepy - Writing video /content/HighwayEnv/scripts/videos/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/HighwayEnv/scripts/videos/rl-video-episode-0.mp4


The environment is a `GoalEnv`, which means the agent receives a dictionary containing both the current `observation` and the `desired_goal` that conditions its policy.

In [ ]:
print("Observation format:", obs)

Observation format: [[ 1.          0.21672086 -0.11365195 -0.0719621  -0.56457394]
 [ 1.          0.19616605 -0.16291884 -0.40538466 -0.48971155]
 [ 1.          0.02279118 -0.4229874  -0.12317721 -1.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]]


There is also an `achieved_goal` that won't be useful here (it only serves when the state and goal spaces are different, as a projection from the observation to the goal space).

Alright! We are now ready to apply the inverse reinforcement learning paradigm.

## Experience collection
First, we randomly interact with the environment to produce a batch of experiences

$$D = \{s_t, a_t, s_{t+1}\}_{t\in[1,N]}$$

In [ ]:
Transition = namedtuple('Transition', ['state', 'action', 'next_state'])

def collect_interaction_data(env, size=1000, action_repeat=2):
    data, done = [], True
    for _ in trange(size, desc="Collecting interaction data"):
        action = env.action_space.sample()
        for _ in range(action_repeat):
            if done:
              previous_obs, info = env.reset()
            obs, reward, done, truncated, info = env.step(action)
            # print("obs:", obs)
            # print("reward:", reward)
            # print("previous_obs:", previous_obs)
            # break
            data.append(Transition(torch.Tensor(previous_obs),
                                   torch.Tensor(action),
                                   torch.Tensor(obs)))
            previous_obs = obs
        # break
    return data

env = gym.make("roundabout-v0")
data = collect_interaction_data(env)
print("Sample transition:", data[0])

Sample transition: Transition(state=tensor([[ 1.0000e+00,  2.0000e-02,  4.5000e-01,  0.0000e+00, -5.3333e-01],
        [ 1.0000e+00, -7.8204e-03,  1.9985e-01,  1.0000e+00,  4.1585e-02],
        [ 1.0000e+00, -1.9743e-01,  1.3646e-01,  4.7218e-01,  6.8316e-01],
        [ 1.0000e+00,  1.0000e+00, -2.0000e-02, -9.4718e-01,  2.2204e-16],
        [ 1.0000e+00, -1.7069e-01, -1.0424e-01, -4.2091e-01,  6.8924e-01]]), action=tensor([1.9135e+23, 4.3833e-41]), next_state=tensor([[ 1.0000e+00,  2.5094e-02,  3.7028e-01,  5.7208e-02, -5.3026e-01],
        [ 1.0000e+00, -1.0341e-01,  2.1679e-01,  6.8140e-01,  4.7472e-01],
        [ 1.0000e+00,  1.3714e-01,  1.4329e-01,  8.3609e-01, -6.5725e-01],
        [ 1.0000e+00, -1.9697e-01,  1.2318e-02, -2.5899e-02,  8.0718e-01],
        [ 1.0000e+00,  1.0000e+00, -2.0000e-02, -9.4718e-01,  2.2204e-16]]))


In [ ]:
! pip install --user git+https://github.com/shubham0704/rl-agents

  Cloning https://github.com/shubham0704/rl-agents to /tmp/pip-req-build-s1h8mlnw
  Running command git clone --filter=blob:none --quiet https://github.com/shubham0704/rl-agents /tmp/pip-req-build-s1h8mlnw
  Resolved https://github.com/shubham0704/rl-agents to commit b8afc1b88b0ba48a45688d70dfb16d60feca46d8
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for rl-agents: filename=rl_agents-1.0.dev0-py3-none-any.whl size=112887 sha256=26484c3ebfe66cecbbb5cd42f9dc35ad5049c5910143e293f49ff358f5eb677f
  Stored in directory: /tmp/pip-ephem-wheel-cache-0b3cyduc/wheels/ff/d2/20/c9b38b3e22061ba79bab253479345cca436635388c292071e6
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=2990b9d626669fcd43d3408afb303a9a7dd8f65663840bdf6a73ba03348b27ce
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built rl-agents docopt


In [ ]:
! git clone https://github.com/shubham0704/rl-agents.git

Cloning into 'rl-agents'...
remote: Enumerating objects: 6381, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 6381 (delta 39), reused 83 (delta 26), pack-reused 6271
Receiving objects: 100% (6381/6381), 1009.86 KiB | 6.27 MiB/s, done.
Resolving deltas: 100% (4489/4489), done.


In [ ]:
%cd rl-agents/scripts/

/content/HighwayEnv/scripts/rl-agents/scripts


In [ ]:
! python experiments.py benchmark configs/RoundaboutEnv/benchmark_robust_control.json \
                      --test --episodes=100 --processes=4

Streaming output truncated to the last 5000 lines.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.get_available_actions to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_available_actions` for environment variables or `env.get_wrapper_attr('get_available_actions')` that will search the reminding wrappers.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.get_available_actions to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_available_actions` for environment variables or `env.get_wrapper_attr('get_available_actions')` that will search the reminding wrappers.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.get_available_actions to get variables from other wrappers is depr

## Build a dynamics model

Dynamics model depends on the nature of environment. Model based RL suffers from the problem of **Model bias**

The model will be accurate only on some region of the state space that was explored and covered in  $D$ . Outside of  $D$ , the model may diverge and hallucinate important rewards. This effect is problematic when the model is used by a planning algorithm, as the latter will try to exploit these hallucinated high rewards and will steer the agent towards unknown (and thus dangerous) regions where the model is erroneously optimistic.

You need to choose from the following:
1. Model-based RL methods
2. Model-free RL methods

## Part 1

List all the known methods you have studied in the class from 1 and 2 and suggest which is the best method you wish to implement for this problem.


1. What is the method? Model-based or Model-free
2. Motivation for choosing the method
3. Problem formulation using your proposed method. Mention metrics (cost function), block-diagram of your pipeline with proper notation and mathematical formulation


## Part 2

Show implementation of your approach and mention results with plots. Add a section to showcase failure cases and mention limitation and future work.


**Example of a dynamics model**

Here is an sample example that uses a model based approach.

We now design a model to represent the system dynamics. We choose  a **structured model** inspired from *Linear Time-Invariant (LTI) systems*

$$\dot{x} = f_\theta(x, u) = A_\theta(x, u)x + B_\theta(x, u)u$$

where the $(x, u)$ notation comes from the Control Theory community and stands for the state and action $(s,a)$. Intuitively, we learn at each point $(x_t, u_t)$ the **linearization** of the true dynamics $f$ with respect to $(x, u)$.

We parametrize $A_\theta$ and $B_\theta$ as two fully-connected networks with one hidden layer.


In [ ]:
env.unwrapped.config

{'observation': {'type': 'Kinematics',
  'absolute': True,
  'features_range': {'x': [-100, 100],
   'y': [-100, 100],
   'vx': [-15, 15],
   'vy': [-15, 15]}},
 'action': {'type': 'DiscreteMetaAction', 'target_speeds': [0, 8, 16]},
 'simulation_frequency': 15,
 'policy_frequency': 1,
 'other_vehicles_type': 'highway_env.vehicle.behavior.IDMVehicle',
 'screen_width': 600,
 'screen_height': 600,
 'centering_position': [0.5, 0.6],
 'scaling': 5.5,
 'show_trajectories': False,
 'render_agent': True,
 'offscreen_rendering': False,
 'manual_control': False,
 'real_time_rendering': False,
 'incoming_vehicle_destination': None,
 'collision_reward': -1,
 'high_speed_reward': 0.2,
 'right_lane_reward': 0,
 'lane_change_reward': -0.05,
 'duration': 11,
 'normalize_reward': True}

In [ ]:
class DynamicsModel(nn.Module):
    STATE_X = 0
    STATE_Y = 1

    def __init__(self, state_size, action_size, hidden_size, dt):
        super().__init__()
        self.state_size, self.action_size, self.dt = state_size, action_size, dt
        A_size, B_size = state_size * state_size, state_size * action_size
        self.A1 = nn.Linear(state_size + action_size, hidden_size)
        self.A2 = nn.Linear(hidden_size, A_size)
        self.B1 = nn.Linear(state_size + action_size, hidden_size)
        self.B2 = nn.Linear(hidden_size, B_size)

    def forward(self, x, u):
        """
            Predict x_{t+1} = f(x_t, u_t)
        :param x: a batch of states
        :param u: a batch of actions
        """
        print(x.shape, u.shape)
        xu = torch.cat((x, u), -1)
        xu[:, self.STATE_X:self.STATE_Y+1] = 0  # Remove dependency in (x,y)
        A = self.A2(F.relu(self.A1(xu)))
        A = torch.reshape(A, (x.shape[0], self.state_size, self.state_size))
        B = self.B2(F.relu(self.B1(xu)))
        B = torch.reshape(B, (x.shape[0], self.state_size, self.action_size))
        dx = A @ x.unsqueeze(-1) + B @ u.unsqueeze(-1)
        return x + dx.squeeze()*self.dt

# dynamics = DynamicsModel(state_size=env.observation_space.shape[0],
#                          action_size=env.action_space.n,
#                          hidden_size=64,
#                          dt=1/env.unwrapped.config["policy_frequency"])
# print("Forward initial model on a sample transition:", dynamics(data[0].state.unsqueeze(0),
#                                                                 data[0].action.unsqueeze(0)).detach())

torch.Size([1, 5, 5]) torch.Size([1, 2])


RuntimeError: ignored

## Scenario1: Leverage dynamics model for planning

We now use the learnt dynamics model $f_\theta$ for planning.
In order to solve the optimal control problem, we use a sampling-based optimization algorithm: the **Cross-Entropy Method** (`CEM`). It is an optimization algorithm applicable to problems that are both **combinatorial** and **continuous**, which is our case: find the best performing sequence of actions.

This method approximates the optimal importance sampling estimator by repeating two phases:
1. **Draw samples** from a probability distribution. We use Gaussian distributions over sequences of actions.
2. Minimize the **cross-entropy** between this distribution and a **target distribution** to produce a better sample in the next iteration. We define this target distribution by selecting the top-k performing sampled sequences.

![Credits to Olivier Sigaud](https://github.com/yfletberliac/rlss2019-hands-on/blob/master/imgs/cem.png?raw=1)

Note that as we have a local linear dynamics model, we could instead choose an `Iterative LQR` planner which would be more efficient. We prefer `CEM` in this educational setting for its simplicity and generality.

## Visualize a few episodes

En voiture, Simone !

In [ ]:
env = gym.make("roundabout-v0", render_mode='rgb_array')
env = record_videos(env)
obs, info = env.reset()

for step in trange(3 * env.config["duration"], desc="Testing 3 episodes..."):
    action = cem_planner(torch.Tensor(obs["observation"]),
                         torch.Tensor(obs["desired_goal"]),
                         env.action_space.shape[0])
    obs, reward, done, truncated, info = env.step(action.numpy())
    if done or truncated:
        obs, info = env.reset()
env.close()
show_videos()

# Mention Limitations of your approach

### Example: Computational cost of planning

At test time, the planning step typically requires **sampling a lot of trajectories** to find a near-optimal candidate, wich may turn out to be very costly. This may be prohibitive in a high-frequency real-time setting. The **model-free** methods which directly recommend the best action are **much more efficient** in that regard.